In [111]:
from pathlib import Path
import yaml

import pandas as pd
import respy as rp
import numpy as np
#params = pd.read_csv("robinson-crusoe.csv")
#options = yaml.safe_load(Path("robinson-crusoe.yaml").read_text())
#simulate = rp.get_simulate_func(params, options)
#df = simulate(params)
np.random.seed(123)
from respy.tests.random_model import generate_random_model
params, options = generate_random_model()
params.loc["meas_error", :] = 0
params.loc["shocks_sdcorr", 'sd_b'] = 1
options["n_periods"] = 1
options["simulation_agents"] = 1
simulate = rp.get_simulate_func(params, options)
df = simulate(params)
df.head()

,Identifier,Period,Choice,Wage,Experience_A,Experience_B,Experience_Edu,Lagged_Choice_1,Type,Nonpecuniary_Reward_A,...,Flow_Utility_Home,Value_Function_A,Value_Function_B,Value_Function_Edu,Value_Function_Home,Shock_Reward_A,Shock_Reward_B,Shock_Reward_Edu,Shock_Reward_Home,Discount_Rate
0,0,0,a,0.627107,0,0,7,b,2,-0.030428,...,-0.584909,0.596679,0.471203,-23282.566425,-0.584909,0.567567,0.536783,0.440311,-0.611009,0.805777


In [127]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [182]:
params, options = rp.get_example_model("kw_94_one", with_data=False)
params

value  \
category            name                                  
delta               delta                        0.9500   
wage_a              constant                     9.2100   
                    exp_edu                      0.0380   
                    exp_a                        0.0330   
                    exp_a_square                -0.0005   
                    exp_b                        0.0000   
                    exp_b_square                 0.0000   
wage_b              constant                     8.4800   
                    exp_edu                      0.0700   
                    exp_b                        0.0670   
                    exp_b_square                -0.0010   
                    exp_a                        0.0220   
                    exp_a_square                -0.0005   
nonpec_edu          constant                     0.0000   
                    at_least_twelve_exp_edu      0.0000   
                    not_edu_last_period      -4000.0000   
nonpec_home         constant                 17750.0000   
shocks_sdcorr       sd_a                         0.2000   
                    sd_b                         0.2500   
                    sd_edu                    1500.0000   
                    sd_home                   1500.0000   
                    corr_b_a                     0.0000   
                    corr_edu_a                   0.0000   
                    corr_edu_b                   0.0000   
                    corr_home_a                  0.0000   
                    corr_home_b                  0.0000   
                    corr_home_edu                0.0000   
meas_error          sd_a                         1.0000   
                    sd_b                         1.0000   
lagged_choice_1_edu edu_ten                      1.0000   
initial_exp_edu     10                           1.0000   
maximum_exp         edu                         20.0000   

                                                                                       comment  
category            name                                                                        
delta               delta                                                      discount factor  
wage_a              constant                                               log of rental price  
                    exp_edu                          return to an additional year of schooling  
                    exp_a                                     return to same sector experience  
                    exp_a_square                   return to same sector, quadratic experience  
                    exp_b                                    return to other sector experience  
                    exp_b_square                  return to other sector, quadratic experience  
wage_b              constant                                               log of rental price  
                    exp_edu                          return to an additional year of schooling  
                    exp_b                                     return to same sector experience  
                    exp_b_square                   return to same sector, quadratic experience  
                    exp_a                                    return to other sector experience  
                    exp_a_square                  return to other sector, quadratic experience  
nonpec_edu          constant                            constant reward for choosing education  
                    at_least_twelve_exp_edu        reward for going to college (tuition, etc.)  
                    not_edu_last_period                        reward for going back to school  
nonpec_home         constant                         constant reward of non-market alternative  
shocks_sdcorr       sd_a                     Element 1,1 of standard-deviation/correlation ...  
                    sd_b                     Element 2,2 of standard-deviation/correlation ...  
      

In [209]:
#options["n_periods"] = 
options["simulation_agents"] = 1

simulate = rp.get_simulate_func(params, options)
df = simulate(params)

In [215]:
crit_func = rp.get_crit_func(params, options, df)
crit_func(params)

Udpagni
Udpagni2
I am in2 [2.44429287 2.18733944 1.46365369 3.10914222 2.12776197 0.2976232
        nan 1.56906127 2.62397697 6.44202895 0.36359537 1.03306582
 1.38448977 1.3482247  0.8531246  5.51178876 1.11977802 0.44519283
 0.43455351 0.62724238 0.9003323  4.36421616 2.15386703 0.78121888
 0.39621923 1.02661604 1.16382804 0.30033806 0.1814797  1.28571891
 0.80324659 2.31721225 3.88258844 0.77353031 0.3958917         nan
 2.52660556 0.19868219        nan 0.52442661]


-433.17556845805075

In [216]:
rslt = pkl.load(open("debug.respy.pkl", "rb"))

In [217]:
np.sqrt(rslt[np.isfinite(rslt)].var())

0.9078752188864495

In [218]:
np.sqrt(np.log(df["Shock_Reward_A"]).var())

0.1962739159832245

In [219]:
df.head(4)

,Identifier,Period,Choice,Wage,Experience_A,Experience_B,Experience_Edu,Lagged_Choice_1,Type,Nonpecuniary_Reward_A,...,Flow_Utility_Home,Value_Function_A,Value_Function_B,Value_Function_Edu,Value_Function_Home,Shock_Reward_A,Shock_Reward_B,Shock_Reward_Edu,Shock_Reward_Home,Discount_Rate
0,0,0,a,35730.354233,0,0,10,edu,0,0.0,...,19339.378043,359632.041963,357359.167314,354740.671575,355897.604045,1.154231,1.293855,-2875.095456,1589.378043,0.95
1,0,1,a,33030.476319,1,0,10,a,0,0.0,...,15994.735451,359785.632075,355550.916466,353404.738337,355021.009062,0.979806,0.909449,-1620.841705,-1755.264549,0.95
2,0,2,a,22809.578540,2,0,10,a,0,0.0,...,21113.346217,365963.047431,357424.006505,357255.343771,362150.922187,1.230656,0.947062,1051.039033,3363.346217,0.95
3,0,3,a,49953.451228,3,0,10,a,0,0.0,...,19701.764892,371101.733280,359693.792346,355528.851153,362442.906912,1.421402,1.068773,-1266.157510,1951.764892,0.95
